In [2]:
#| default_exp diffusion_laziness
# Diffusion Curvature utils
from diffusion_curvature.utils import *
from diffusion_curvature.datasets import *
# Python necessities
import numpy as np
import jax
import jax.numpy as jnp
from fastcore.all import *
import matplotlib.pyplot as plt
# Notebook Helpers
from nbdev.showdoc import *
from tqdm.notebook import trange, tqdm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Diffusion Laziness Estimators
> What's the shape of this diffusion?

# Wasserstein Spread of Diffusion

In [3]:
#|export
import jax.numpy as jnp
from jax import jit

@jit
def wasserstein_spread_of_diffusion(
                D, # manifold geodesic distances
                Pt, # powered diffusion matrix/t-step ehat diffusions
                ):
        """
        Returns how "spread out" each diffusion is, with wasserstein distance
        Presumes that the manifold distances have been separately calculated
        """
        return jnp.sum(D * Pt, axis=-1)

### Benchmarking

In [25]:
D = np.random.rand(1000,1000)
Pt = np.random.rand(1000,1000)
Pt = Pt / np.sum(Pt, axis=1)[:,None]

In [7]:
%%timeit
wasserstein_spread_of_diffusion(D,Pt)

898 µs ± 8.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
key = jax.random.PRNGKey(0)
Djax = jax.random.normal(key, (1000, 1000))
key = jax.random.PRNGKey(10)
Ptjax = jax.random.normal(key, (1000, 1000))

In [11]:
jnp.allclose(Djax,Ptjax)

Array(False, dtype=bool)

In [12]:
%%timeit
wasserstein_spread_of_diffusion(Djax,Ptjax)

20.8 µs ± 429 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Wow, it's nearly two orders of magnitude faster when using jax arrays.

# Entropy of Diffusion

In [14]:
#|export
import jax.scipy
import jax.numpy as jnp
# def entropy_of_diffusion(Pt):
#         """
#         Returns the pointwise entropy of diffusion from the powered diffusion matrix in the input
#         Assumes that Pt sums to 1
#         """
#         Pt = (Pt + 1e-10) /(1 + 1e-10*Pt.shape[0]) # ensure, for differentiability, that there are no zeros in Pt, but that it still sums to 1.
#         entropy_elementwise = jax.scipy.special.entr(Pt)
#         entropy_of_rows = jnp.sum(entropy_elementwise, axis=-1)
#         # normalize so max value is 1
#         entropy_of_rows = entropy_of_rows / (-jnp.log(1/Pt.shape[0]))
#         return entropy_of_rows
def entropy_of_diffusion(Pt, epsilon=1e-10):
        """
        Returns the pointwise entropy of diffusion from the powered diffusion matrix in the input
        Assumes that Pt sums to 1
        """
        # Pt = (Pt + 1e-10) /(1 + 1e-10*Pt.shape[0]) # ensure, for differentiability, that there are no zeros in Pt, but that it still sums to 1.
        entropy_elementwise = jax.scipy.special.entr(Pt)
        entropy_of_rows = jnp.sum(entropy_elementwise, axis=-1)
        # normalize so max value is 1
        # entropy_of_rows = entropy_of_rows / (-jnp.log(1/jnp.sum(Pt>epsilon, axis=-1)))
        return entropy_of_rows

In [27]:
from scipy.stats import entropy
assert jnp.allclose(entropy_of_diffusion(Pt),entropy(Pt,axis=1))